In [1]:
import warnings
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')


rs = 132
np.random.seed(1)
dataset = pd.read_csv('./data/dataset.csv')
# '营业利润增长率'/'应收账款周转率'
features = [
    '净资产收益率', '投入资本回报率', '成本费用利润率',
    '总资产周转率', '流动资产周转率', '存货周转率', '资产负债率',
    '速动比率', '现金流动负债比率', '营业总收入增长率', '总资产增长率',
    '员工收入增长率(%)', '发明专利', '研发人员占比(%)', '研发营收比(%)',
    '股权集中度(%)', '两权分离率(%)'
]
label_name = '因子得分'
# 建立企业id与名称的索引
index_map = dataset["股票简称"].to_dict()
# 获取数据集和标签值
y_test : pd.Series = dataset[label_name]
X : pd.DataFrame = dataset[features].copy(deep=True).astype("float")
# 数据预处理：1.极差标准化；2.数据集划分。
X_train, X_test, y_train, y_test = train_test_split(X, y_test, test_size=0.2, random_state=rs)
scaler = MinMaxScaler()
scaler.fit(X_train)
# scaler.fit(X)

# X_train_s = scaler.transform(X)
X_train_s = scaler.transform(X_train)
X_test_s = scaler.transform(X_test)

In [2]:
from config import r2_score
from tensorflow.keras import models, utils, layers, regularizers, callbacks, optimizers


# Sequential 模型适用于普通层堆栈 其中，每层只有一个 input Tensor 和一个 Output Tensor。
model = models.Sequential()
# model.add(keras.Input(shape=(X_train_s.shape[1], )))
model.add(layers.Dense(units=15, activation='relu', name="layer1", input_shape=(X_train_s.shape[1], ),
                               kernel_regularizer=regularizers.l2(0.02)))
model.add(layers.Dropout(0.08))
# 增加输出层
model.add(layers.Dense(units=1, name="output"))
optimizer = optimizers.Adam(learning_rate=9e-3)
model.compile(optimizer=optimizer, loss='mse', metrics=['mse', r2_score, 'mae'])
# 查看模型结构
# utils.plot_model(model, "./assert/feature_importance/bp_model_structure.png", show_shapes=True)
# model.summary()
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
hist = model.fit(X_train_s, y_train, validation_split=0.3, epochs=300, batch_size=30, shuffle=False, verbose=0,
                         callbacks=[early_stopping])
# validation_split是训练集验证集拆分，epochs代表训练300轮，batch_size代表在批量梯度下降时每次选择16个样本，shuffle代表在训练过程中不会将数据反复打乱
# verbose：日志显示，0为不在标准输出流输出日志信息，1为输出进度条记录，2为每个epoch输出一行记录。
s = model.evaluate(X_test_s, y_test, verbose=0)
print(f'测试集mse得分:{s[1]:.3%}，R^2得分{s[2]:.3%}')
"""
#### 关于损失函数
MSE（Mean Squared Error）均方误差是回归问题中常用的损失函数之一。MSE损失函数可以用于量化预测值与真实值之间的差距，并且对误差进行平方处理，使得较大的误差对损失函数的影响更加显著。其优势在于由于对结果进行了平方处理，使得对异常值表现比较敏感，因此可以比较准确的反映出模型预测的准确性，很适合用于回归问题。</br>缺点在于，对异常值过于敏感，易因异常值原因导致模型偏差较大，在计算损失时，选择了等权重方法，现实中样本的重要性可能不同。
其定义为：
$$ MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2 $$
其中，$y_i$为真实值，$\hat{y}_i$为模型预测值，$n$为样本数量。
**其他损失函数**
- 加权MSE：适用于针对样本差异性大的情形，加权MSE可以减少异常值的影响，提高模型的鲁棒性。$np.mean(weights*(y_{true}-y_{pred})**2)$
- MAE：取绝对值而不是取平方，优点是完美表达损失，不易受异常值（偏差）影响，稳健型强。缺点在于不可导，无法使用梯度下降算法，故优化速度慢。
- Huber：中和了MSE和MAE，具有一定的抗异常值的特性，但同时中和了其缺点，包含大量的额外参数。
- Log-cosh: 在损失函数为较大值时收敛速度较快，对异常值有一定容忍度。适用于误差比较大的数据集，其计算相对复杂。
</br>
</br>
#### 可用的优化器
SGD（随机梯度下降）：最基本的优化器，每次只使用一个样本来更新权重。</br>
Momentum：结合了动量概念，加速SGD在相关方向上的收敛，同时抑制震荡。</br>
Adam：自适应学习率的优化器，结合了Momentum和RMSProp的特点。</br>
Adagrad：对每个参数使用不同的学习率，但随着迭代次数的增加，学习率可能会变得非常大。</br>
Adadelta：改进版的Adagrad，使用动态学习率。</br>
RMSProp：与Adadelta类似，但使用指数移动平均来计算方差。</br>
FTRL：适用于在线学习问题的优化器。</br>
Proximal Gradient Descent：用于求解约束优化问题。</br>
FTRL-Proximal Gradient Descent：结合了FTRL和Proximal Gradient Descent。</br>
其中，SGD和Adam适用于简单的回归和分类问题，Adam在复杂的任务或大规模数据集表现较好，Adagrad、Adadelta或RMSProp能够更精细的控制学习率，FTRL和Proximal Gradient Descent则适用于在线学习或分布式系统，如果问题包含特定的约束条件或问题结构，如稀疏性或低维性等，则需要选择特定的优化器，如FTRL或Proximal Gradient Descent。
"""
model.save("./assert/bp_keras_model.keras")
pd.DataFrame(hist.history).to_csv('./assert/bp_model_loss.csv')

2024-12-12 03:03:13.244591: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-12 03:03:13.254921: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733943793.266507   10013 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733943793.270203   10013 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-12 03:03:13.284419: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

测试集mse得分:0.311%，R^2得分97.325%


In [3]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from config import table_translate

result = pd.DataFrame(data=["ANN", s[2], s[1], s[3]], index=["model_name", "r2_score", "mse", "mae"]).T
model = LinearRegression(n_jobs=-1)
model.fit(X_train_s, y_train)
y_pred = model.predict(X_test_s)
result.loc[result.shape[0]] = ["OLS Regression", r2_score(y_test, y_pred), mean_squared_error(y_test, y_pred),mean_absolute_error(y_test, y_pred)]
best_params={'ccp_alpha': 0.00039393774166455076, 'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2}
model = DecisionTreeRegressor(random_state=123, **best_params)
model.fit(X_train_s, y_train)
y_pred = model.predict(X_test_s)
result.loc[result.shape[0]] = ["Decision Tree", r2_score(y_test, y_pred), mean_squared_error(y_test, y_pred),mean_absolute_error(y_test, y_pred)]
best_params =  {'max_depth': 11, 'max_features': 9, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 69}
model = RandomForestRegressor(**best_params, verbose=0, n_jobs=-1, random_state=123)
model.fit(X_train_s, y_train)
y_pred = model.predict(X_test_s)
result.loc[result.shape[0]] = ["Random Forest", r2_score(y_test, y_pred), mean_squared_error(y_test, y_pred),mean_absolute_error(y_test, y_pred)]

best_params = {'colsample_bytree': 0.77, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 10, 'min_child_weight': 2, 'n_estimators': 63, 'subsample': 0.37}
other_params = {"objective": 'reg:squarederror', "reg_alpha": 0, "reg_lambda": 1, "scale_pos_weight": 1}
model = XGBRegressor(**best_params, **other_params, verbose=0, n_jobs=-1, random_state=123)
model.fit(X_train_s, y_train)
y_pred = model.predict(X_test_s)
result.loc[result.shape[0]] = ["XGBoost", r2_score(y_test, y_pred), mean_squared_error(y_test, y_pred),mean_absolute_error(y_test, y_pred)]

best_params = {"n_estimators": 100, "objective": 'mse', "min_split_gain": 0, 'reg_alpha': 0, 'reg_lambda': 0, "force_col_wise": True, 'max_depth': 12, "learning_rate": 0.1, 'colsample_bytree': 0.25, "subsample": 0.8, 'min_child_samples': 7, "num_leaves": 30}
model = LGBMRegressor(**best_params, n_jobs=-1, random_state=123, verbosity=-1)
model.fit(X_train_s, y_train)
y_pred = model.predict(X_test_s)
result.loc[result.shape[0]] = ["LightGBM", r2_score(y_test, y_pred), mean_squared_error(y_test, y_pred),mean_absolute_error(y_test, y_pred)]

params ={"iterations": 300, "learning_rate": 0.1, "depth": 3, "l2_leaf_reg": 1, 'bagging_temperature': 0, "border_count": 64}
model = CatBoostRegressor(**params, random_state=123, train_dir=None, verbose=0)
model.fit(X_train_s, y_train, verbose=0
          )
y_pred = model.predict(X_test_s)
result.loc[result.shape[0]] = ["CatBoost", r2_score(y_test, y_pred), mean_squared_error(y_test, y_pred),mean_absolute_error(y_test, y_pred)]
# result.to_csv('./assert/tables/ml_model_performance.csv', index=False)
table_translate(result, filename='机器学习性能评估')
result

,model_name,r2_score,mse,mae
0,ANN,0.973253,0.003113,0.040779
1,OLS Regression,1.0,0.0,0.0
2,Decision Tree,0.75058,0.028969,0.128571
3,Random Forest,0.882841,0.013607,0.081851
4,XGBoost,0.935021,0.007547,0.059592
5,LightGBM,0.91969,0.009328,0.065122
6,CatBoost,0.956862,0.00501,0.045744


In [ ]:
import shap
from types import MethodType
# import matplotlib.pyplot as plt
from sklearn.inspection import permutation_importance
# plt.rcParams['font.sans-serif'] = ['SimHei'] # 用来正常显示中文标签
# plt.rcParams['axes.unicode_minus'] = False # 用来正常显示负号


score = lambda self, X, y: self.evaluate(X, y, verbose=0)[1]
# (1)权重特征重要度；
model = models.load_model("./assert/bp_keras_model.keras")
model.score = MethodType(score, model)
layer_weights = model.layers[0].get_weights()[0]
feature_importance = np.mean(np.abs(layer_weights), axis=1)  # 取绝对值的平均值

# (2)对训练集进行置换重要性分析：计算时间非常长，大概需要4分钟
# result = permutation_importance(model, X_train_s, y_train, n_repeats=200, random_state=42, n_jobs=-1)
# # n_repeats=20：进行50次随机替换
# # Bunch_result:[importance(置换重要度),importance_mean（置换重要度均值）,importance_std（置换重要度标准差）]
# result.importances_mean
# # Bunch_index = Bunch_result.importances_mean.argsort()
# ax.boxplot(Bunch_result.importances[Bunch_index].T, vert=False, labels=nlabels)  #  labels=Xnames

# 检验神经网络模型对 2010 年 164 家上市公司经营绩效的拟合效果，计算神经网络绩效评 价得分与经营绩效实际得分的 Pearson 相关系数，其值为 0.966，均方误差为 0.0013，平均绝 对误差为 0.027，平均相对误差为 3.89%，由此可知 BP 神经网络评价模型对 2010 年民营制造 业上市公司经营绩效的拟合效果非常好。从排名差异来看，由表 4.4 可知，排名前五的上市 公司实际排名和预测排名完全一致，排名后五的上市公司两者排名相差较小；检验排名的次 序相关程度，计算 Spearman 次序相关系数，其值为 0.922，说明预测排名与实际排名基本一 致，因此可以认为 BP 神经网络评价模型对 2010 年 164 家民营制造业上市公司经营绩效的拟 合效果非常好。
shap.initjs()
explainer = shap.DeepExplainer(model, X_train_s)
shap_values = explainer.shap_values(X_test_s)
_shap_values = shap_values.T[0].T

# fig, ax = plt.subplots(figsize=(8, 6), dpi=300)

# bars = ax.barh(features, feature_importance, left=0, height=0.5, color='skyblue')
shap.summary_plot(_shap_values, feature_names=features, sort=False)  # , plot_type="bar", plot_type='bar'

# 设置标题
# plt.title('各类别数据展示')
# ax.set_xlabel('重要度')
# ax.set_ylabel('特征')
# plt.grid(axis='x', alpha=0.5, linestyle='--')

# , transparent=True, bbox_inches='tight'
# plt.show()

In [ ]:
"""'
任务：
1.如果计算时间过长，可以先使用 RandomizedSearchCV 进行粗略搜索，再用 GridSearchCV 精调。
2.对于 CatBoost 和 LightGBM，可以通过设置 early_stopping_rounds 参数加快训练。
3.特征重要性分析：
    •XGBoost: model.best_estimator_.feature_importances_
    •LightGBM: model.best_estimator_.booster_.feature_importance()
    •CatBoost: model.best_estimator_.get_feature_importance()


# 检验神经网络模型对 2010 年 164 家上市公司经营绩效的拟合效果，计算神经网络绩效评 价得分与经营绩效实际得分的 Pearson 相关系数，其值为 0.966，均方误差为 0.0013，平均绝 对误差为 0.027，平均相对误差为 3.89%，由此可知 BP 神经网络评价模型对 2010 年民营制造 业上市公司经营绩效的拟合效果非常好。从排名差异来看，由表 4.4 可知，排名前五的上市 公司实际排名和预测排名完全一致，排名后五的上市公司两者排名相差较小；检验排名的次 序相关程度，计算 Spearman 次序相关系数，其值为 0.922，说明预测排名与实际排名基本一 致，因此可以认为 BP 神经网络评价模型对 2010 年 164 家民营制造业上市公司经营绩效的拟 合效果非常好。
"""